# Kурс Open Data Science

## Соревнование "Прогноз популярности статьи на Хабре"

<b>Волков Дмитрий</b>

Ссылка на конкурс: https://inclass.kaggle.com/c/howpop-habrahabr-favs

## 1. Загрузка данных в json

<b>Сначала важный технический момент.</b>

Я создавал 2 пары json файлов: train и test включающие содержание статей и не включающие (поле content). Работать с содержанием имеет смысл, только если у вас есть 16 Гб оперативной памяти. Иначе это будет очень долго и не факт что успешно (особенно под Windows). 

По умолчанию всё настроено на использование содержания. Если хотите изменить это - измените значение переменной ниже.

Модель без использования содержания статей теряет примерно 0.05 и дает результат на прайват в районе 0.398

In [6]:
# Сделайте свой выбор сразу :)
use_content = True

Ниже приведены пути к папкам с исходными данными и файлам которые будут сгенерированы: создайте у себя такую же структуру или поправьте пути. Также создайте папку /results для записи сабмишена.

In [7]:
# Пути к файлам
# корневая папка с исходными данными
data_root = "./data/new"
# папка со всеми исходными json файлами для трейна
train_dir = data_root + "/train"
# папка со всеми исходными json файлами для теста
test_dir = data_root + "/test"

# файлы которые будут созданы при загрузке данных
train_file_content = data_root + "/train_new.json"
test_file_content = data_root + "/test_new.json"
target_csv = data_root + "/train_target.csv"

# имена json без содержания статей
train_file_light = data_root + "/train_new_light.json"
test_file_light = data_root + "/test_new_light.json"

# сабмишн
submission_file = './results/sub_ridge.csv'

In [8]:
import glob, os
import pandas as pd
import re
import datetime

In [9]:
numbers = re.compile(r'(\d+)')

def preprocess_json_row(json_df):
    # формируем из 'hubs' [id] и [title] -> списки
    row = json_df.loc[0, 'hubs']
    hub_id_list, hub_title_list, hub_url_list = [], [], []
    #строка - это список словарей. Получаем нужные нам поля
    for dct in row:
        hub_id_list.append(dct['id'])
        hub_title_list.append(dct['title'])
        hub_url_list.append(dct['url'])

    json_df.loc[0, 'hub_id'] = ', '.join(hub_id_list)
    json_df.loc[0, 'hub_title'] = ', '.join(hub_title_list)

    return json_df

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def create_json(is_train=True, use_content=False, firstN = 0):
    current_dir = os.getcwd()
    if is_train:
        os.chdir(train_dir)
    else:
        os.chdir(test_dir)

    train_df = pd.DataFrame()
    train_df_all = pd.DataFrame()
    for (i, file) in enumerate(sorted(glob.glob("*.json"), key=numericalSort)):
        json_df = pd.read_json(file, lines=True)
        json_df = preprocess_json_row(json_df)
        train_df = train_df.append(json_df, ignore_index=True)

        if (i + 1) % 1000 == 0:
            print(file)
            print('{} {} files processed'.format(datetime.datetime.now().time(), i + 1))

        # производительность df.append сильно падает после нескольких тыс вызовов. 
        # будем пересоздавать train_df каждые 5000 строк, а обработанное складывать в train_df_all
        if (i + 1) % 5000 == 0:
            # заполним content_len и почистим content если надо
            train_df['content_len'] = train_df['content'].astype(str).apply(len)
            if not use_content:
                train_df['content'] = ''
            #сохраним в all
            train_df_all = train_df_all.append(train_df, ignore_index=True)
            # пересоздадим
            del train_df
            train_df = pd.DataFrame()

    # заполним content_len и почистим content если надо
    train_df['content_len'] = train_df['content'].astype(str).apply(len)
    if not use_content:
        train_df['content'] = ''
    # сохраним в all
    train_df_all = train_df_all.append(train_df, ignore_index=True)
    
    # дополнительно преобразуем часть столбцов
    train_df_all['published'] = train_df_all['published'].apply(lambda x: x['$date'])
    train_df_all['author_nickname'] = train_df_all['author'].apply(lambda x: x['nickname']).fillna('-')
    train_df_all['author_name'] = train_df_all['author'].apply(lambda x: x['name']).fillna('-')
    train_df_all['author_url'] = train_df_all['author'].apply(lambda x: x['url']).fillna('-')

    # добавляем 'flags', 'tags' -> список
    train_df_all['flags'] = train_df_all['flags'].apply(lambda x: ', '.join(x))
    train_df_all['tags'] = train_df_all['tags'].apply(lambda x: ', '.join(x))

    train_df_all = train_df_all.drop(['hubs'], axis=1)
    os.chdir(current_dir)

    # пишем
    train_file = train_file_content if use_content else train_file_light
    test_file = test_file_content if use_content else test_file_light
    
    if is_train:
        target = pd.read_csv(target_csv, index_col='_id')
        train_df_all['favs_lognorm'] = train_df_all['_id'].map(target['favs_lognorm'])
        train_df_all.to_json(train_file)
        print('{} created'.format(train_file))
    else:
        train_df_all.to_json(test_file)
        print('{} created'.format(test_file))        

    return

Создадим 2 json файла: для train и для test. Это может занять минут 40-50

In [ ]:
if use_content:
    create_json(is_train=False, use_content=True)
    create_json(is_train=True, use_content=True)
else:
    # для генерации json без содержания статей
    create_json(is_train=False, use_content=False)
    create_json(is_train=True, use_content=False)

## 2. Модель

In [11]:
import pandas as pd
import numpy as np
import scipy
import os
import datetime

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

import re

seed = 42

In [12]:
cfg = {
    # используем ли содержание статей (сильно всё замедляет, но добавляет ~0.05 к результату)
    # чтобы запустить модель без использования содержания статей, сначала нужно дополнительно сгенерить соответствующую пару json
    'LOAD_CONTENT': use_content, 
    
    # с какой даты начинается train. Всё более старое не используем для обучения
    'CUT_DATE': '2016-01-01' 
}

REGEX_TAGS = re.compile(r",\s*")
REGEX_HUBS = re.compile(r",|/\s*")

frm = {
    '.' : '',
    '!' : '',
    "'" : '',
    '"' : '',
    '-' : '',
    '#' : '',
    '*' : '',
    ' ' : ''
}
TRANS_TABLE = str.maketrans(frm)

def tokenize_tags(text):
    return [tok.translate(TRANS_TABLE).lower() for tok in REGEX_TAGS.split(text)]

def tokenize_hubs(text):
    return [tok.translate(TRANS_TABLE).lower() for tok in REGEX_HUBS.split(text)]

def preprocess_data(X):
    X['published'] = X['published'].apply(lambda x: pd.to_datetime(x, yearfirst=True))
    X['content'] = X['content'].astype(str).fillna('-')
    X['content_len'] = X['content_len'].fillna(1)
    X['title'] = X['title'].astype(str).fillna('-')
    X['author_name'] = X['author_name'].fillna('-')
    X['author_nickname'] = X['author_nickname'].fillna('-')
    #X['author_url'] = X['author_url'].apply(lambda x: '/'.join(x.split('/')[3:]) if not pd.isnull(x) else '')

    return X

def create_features(X, feature_list):
    #не портим переданный список - сделаем копию
    fl = feature_list.copy()
    while fl:
        feature = fl.pop(0)
        if feature == 'polling':
            X.ix[~pd.isnull(X[feature]), feature] = 1
            X[feature] = X[feature].fillna(0).astype(int)

        elif feature == 'start_month':
            X[feature] = X['published'].apply(lambda ts: 100 * ts.year + ts.month)

        elif feature == 'year':
            X[feature] = X['published'].apply(lambda ts: ts.year)

        elif feature == 'month':
            X[feature] = X['published'].apply(lambda ts: ts.month)

        elif feature == 'hour':
            X[feature] = X['published'].apply(lambda ts: ts.hour)

        elif feature == 'hour_min':
            X[feature] = X['published'].apply(lambda ts: ts.hour*100 + ts.minute)

        elif feature == 'morning':
            X[feature] = X['hour'].apply(lambda h: int(h <= 11))

        elif feature == 'dayofweek':
            X[feature] = X['published'].apply(lambda ts: ts.isoweekday())

        elif feature == 'isweekend':
            X[feature] = X['dayofweek'].apply(lambda day: 1 if day in [6, 7] else 0)

        elif feature == 'isweekend_ev':
            X[feature] = X['published'].apply(lambda ts: 1 if ((ts.isoweekday() == 6 or ts.isoweekday() == 7) and ts.hour >= 19) else 0)

        elif feature == 'week_hour':
            X[feature] = X['published'].apply(lambda ts: ts.weekday()*24 + ts.hour)

        elif feature == 'content_len_sqrt':
            X[feature] = X['content_len'].apply(lambda x: np.sqrt(x + 1))

        elif feature == 'content_len_log':
            X[feature] = X['content_len'].apply(lambda x: np.log2(x+1))

        elif feature == 'content_len_log**2':
            X[feature] = X['content_len_log'].apply(np.square)

        elif feature == 'content_len_log**3':
            X[feature] = np.power(X['content_len_log'], 3)

        elif feature == 'content_len_loglog':
            X[feature] = X['content_len_log'].apply(lambda x: np.log(x + 1))

        elif feature == 'n_title_length':
            X['title'] = X['title'].astype(str).fillna('-')
            X['n_title_length'] = X['title'].apply(len)

        elif feature == 'n_title_words':
            X[feature] = X['title'].apply(lambda s: np.log2(len(s.split()) + 1))

        elif feature == 'n_title_words_log':
            X[feature] = X['title'].apply(lambda s: np.log2(len(s.split()) + 1))

        else:
            raise Exception('Feature "{}" is not supported'.format(feature))
        print('Feature {} created'.format(feature))
    return X

def print_some_from_vect(v):
    """ Для отладки. Выводим первые 150, последние 150 и 10000:10150 элементы объекта v
        v - любой Vectorizer
    """
    vlen = len(v.vocabulary_)
    print(vlen)
    vec = np.zeros(vlen); vec[:150] = 1; print(v.inverse_transform(vec))
    vec = np.zeros(vlen); vec[10000:10150] = 1; print(v.inverse_transform(vec))
    vec = np.zeros(vlen); vec[-150:] = 1; print(v.inverse_transform(vec))
    return

def vectorize_and_scale2(X_train, X_test, categorical_feat_list, features_to_scale=None, features_to_add=None, use_content = False):
    print('{} vectorize_and_scale2 started'.format(datetime.datetime.now().time()))

    full_df = pd.concat([X_train[categorical_feat_list], X_test[categorical_feat_list]])
    oh = OneHotEncoder()
    oh.fit(full_df[categorical_feat_list])
    X_train_cat = oh.transform(X_train[categorical_feat_list])
    X_test_cat = oh.transform(X_test[categorical_feat_list])
    print('OneHotEncoder completed')

    '''
    ## tags
    v = CountVectorizer(analyzer='word', tokenizer=tokenize_tags, ngram_range=(1, 1))
    X_train_tags = v.fit_transform(X_train['tags'].fillna('-'))
    X_test_tags = v.transform(X_test['tags'].fillna('-'))
    print('Count vectorizing tags completed')
    '''

    ## hub_id
    v = CountVectorizer(analyzer='word', tokenizer=tokenize_hubs, ngram_range=(1, 1))
    X_train_hub_id = v.fit_transform(X_train['hub_id'].fillna('-'))
    X_test_hub_id = v.transform(X_test['hub_id'].fillna('-'))
    print('Count vectorizing hub_id completed')
    #print_some_from_vect(v)

    ## flags
    v = CountVectorizer(analyzer='word', tokenizer=tokenize_tags, ngram_range=(1, 1))
    X_train_flags = v.fit_transform(X_train['flags'].fillna('-'))
    X_test_flags = v.transform(X_test['flags'].fillna('-'))
    print('Count vectorizing flags completed')

    ## title
    v = TfidfVectorizer(analyzer='word', min_df=2, max_df=0.3, ngram_range=(1, 2), sublinear_tf=True)
    X_train_title = v.fit_transform(X_train['title'])
    X_test_title = v.transform(X_test['title'])
    print('Title word vectorizing vocabulary size: ', len(v.vocabulary_))
    print('Title word vectorizing completed')

    v = TfidfVectorizer(analyzer='char', ngram_range=(1, 6), sublinear_tf=True)
    X_train_title_ch = v.fit_transform(X_train['title'])
    X_test_title_ch = v.transform(X_test['title'])
    print('Title char vectorizing vocabulary size: ', len(v.vocabulary_))
    print('Title char vectorizing completed')

    ## hub_title
    ''' 
    v = TfidfVectorizer(analyzer='word', min_df=2, max_df=0.3, ngram_range=(1, 2), sublinear_tf=True)
    X_train_hub_title = v.fit_transform(X_train['hub_title'])
    X_test_hub_title = v.transform(X_test['hub_title'])
    print('Hub word vectorizing completed')
    '''
    
    v = TfidfVectorizer(analyzer='char', ngram_range=(1, 4), sublinear_tf=True)
    X_train_hub_title_ch = v.fit_transform(X_train['hub_title'])
    X_test_hub_title_ch = v.transform(X_test['hub_title'])
    print('Hub char vectorizing vocabulary size: ', len(v.vocabulary_))
    print('Hub char vectorizing completed')

    ## content
    if use_content:
        ngram_low=1; ngram_high=2; min_df=2; max_df=0.3
        print('{} Content word vectorizing started.\n ngram_range=({},{}), min_df={}, max_df={}'.format(
            datetime.datetime.now().time(), ngram_low, ngram_high, min_df, max_df))
        v = TfidfVectorizer(min_df=min_df, max_df= max_df, ngram_range=(ngram_low, ngram_high), sublinear_tf=True)
        X_train_content = v.fit_transform(X_train['content'])
        X_test_content = v.transform(X_test['content'])

        print('Content word vocabulary size: ', len(v.vocabulary_))
        print('{} Content word vectorizing completed'.format(datetime.datetime.now().time()))

        ngram_low=1; ngram_high=3
        print('{} Content char vectorizing started. ngram_range=({},{})'.format(
            datetime.datetime.now().time(), ngram_low, ngram_high))
        v = TfidfVectorizer(analyzer='char', ngram_range=(ngram_low, ngram_high), sublinear_tf=True)
        X_train_content_ch = v.fit_transform(X_train['content'])
        X_test_content_ch = v.transform(X_test['content'])

        print('Content char vocabulary size: ', len(v.vocabulary_))
        print('{} Content char vectorizing completed'.format(datetime.datetime.now().time()))

    # Стыкуем Vectorizers и OneHot
    X_train_sparse = scipy.sparse.hstack([
        X_train_cat, X_train_hub_id, X_train_flags,
        X_train_title, X_train_title_ch, X_train_hub_title_ch])  #X_train_hub_title, X_train_feats, X_train_tags, X_train_flags,
    X_test_sparse = scipy.sparse.hstack([
        X_test_cat, X_test_hub_id, X_test_flags,
        X_test_title, X_test_title_ch, X_test_hub_title_ch])  # X_test_hub_title, X_test_feats,  X_test_tags, X_test_flags,

    if use_content:
        X_train_sparse = scipy.sparse.hstack([X_train_sparse, X_train_content, X_train_content_ch])
        X_test_sparse = scipy.sparse.hstack([X_test_sparse, X_test_content, X_test_content_ch])

    # Стыкуем отнормированные фичи
    if features_to_scale:
        X_train_sparse = scipy.sparse.hstack([
            X_train_sparse, StandardScaler().fit_transform(X_train[features_to_scale])])
        X_test_sparse = scipy.sparse.hstack([
            X_test_sparse, StandardScaler().fit_transform(X_test[features_to_scale])])

    # Стыкуем остальные фичи
    if features_to_add:
        X_train_sparse = scipy.sparse.hstack([X_train_sparse, X_train[features_to_add]])
        X_test_sparse = scipy.sparse.hstack([X_test_sparse, X_test[features_to_add]])

    return X_train_sparse, X_test_sparse

def encode_categorial_features(train_df, test_df, cat_feature_list):
    full_df = pd.concat([train_df.drop('favs_lognorm', axis=1), test_df])
    for f in cat_feature_list:
        lb = LabelEncoder()
        lb.fit(full_df[f].values.flatten())
        train_df[f] = lb.transform(train_df[f])
        test_df[f] = lb.transform(test_df[f])

    return train_df, test_df

In [ ]:
try:
    print('{} Начали'.format(datetime.datetime.now().time()))
    if cfg['LOAD_CONTENT']:
        train_df = pd.read_json(train_file_content)
        test_df  = pd.read_json(test_file_content)
    else:
        train_df = pd.read_json(train_file_light)
        test_df  = pd.read_json(test_file_light)

    train_df = preprocess_data(train_df)
    test_df = preprocess_data(test_df)

    # Отрежем старые данные
    if cfg['CUT_DATE']:
        train_df = train_df[train_df['published'] >= pd.to_datetime(cfg['CUT_DATE'])]
    
    # генерим дополнительные фичи
    new_features = ['dayofweek', 'content_len_log', 'content_len_log**2', 'isweekend_ev']
    train_df = create_features(train_df, new_features)
    test_df = create_features(test_df, new_features)

    ### Все фичи, которые будем использовать помимо фич для Vectorizers
    # c масштабированием
    features_to_scale = ['isweekend_ev', 'content_len', 'content_len_log', 'content_len_log**2']
    # без масштабирования
    features_to_add = []
    # категориальные
    categorical_feat_list = ['author_nickname', 'domain', 'dayofweek']

    train_df, test_df = encode_categorial_features(train_df, test_df, categorical_feat_list)

    print('{} PREDICT начат'.format(datetime.datetime.now().time()))
    # Целевые веременные
    y_train = train_df['favs_lognorm']
    train_df.drop('favs_lognorm', axis=1)

    # Соберем всё вместе
    X_train_sparse, X_test_sparse = vectorize_and_scale2(
        train_df,
        test_df,
        categorical_feat_list,
        features_to_scale,
        features_to_add,
        cfg['LOAD_CONTENT']
    )
    print('{} PREDICT. Данные загрузили. Размер тренировочной выборки {}'.format(datetime.datetime.now().time(), X_train_sparse.shape))

    alpha = 1
    model = Ridge(alpha=alpha, random_state=seed)
    model.fit(X_train_sparse, y_train)
    test_preds = model.predict(X_test_sparse)
    print('{} PREDICT завершен'.format(datetime.datetime.now().time()))

    # Готовим submission 
    submission = pd.DataFrame()
    submission['_id'] = test_df._id.values
    submission['favs_lognorm'] = test_preds
    submission.to_csv(submission_file, index=False)
    print('WRITE_SUBMISSION. Файл записан')
    
    # работает только на Mac :)
    os.system('say "your program has finished"')

except:
    os.system('say "something has gone wrong"')
    raise

## 3. Заключение 

Что больше всего помогало улучшению результата:
- отрезать все старые данные. 
 Я сильно не оптимизировал, попробовал разные варианты с шагом полгода, лучше всего оказалось начинать с 01.01.2016. Подозреваю, что старые статьи отличаются по содержанию, плюс по старым данным довольно много пропусков и отличается структура в нескольких фичах: хабы, флаги. Возможно и сценарии использования  избранного в первые годы существования Хабра отличались. Это дало в районе +0.15 к результату
- подобрать параметры для tf-idf. Это касается и содержания статей и их заголовков и хабов. Подбор параметров довольно сильно улучшил результат, в частности изменение значения min_df c 3 (значение из бэйзлайна в 4 домашке) на 2. 
- фичи производные от длины статьи. Я попробовал разные варианты, в итоге использовал саму длину, log длины и log^2 от длины. 